In [1]:
import torch
from descriptor import DesNet
from config_profile import args
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

def defineModel(cnnType):
    dic = {"CNN1": 1, "CNN2": 2, "CNN3": 3}
    cnn_model = DesNet(dic[cnnType])
    if args.cuda:
        cnn_model.cuda()
    return cnn_model


batch, cnnList = "512", ["CNN1","CNN2","CNN3"]
patches_dir = "wangchih_patchs.pt"
output_dir = "wangchih_descriptions.pt"
outList = []

# Load patches data
patches = torch.load(patches_dir)
patches = patches.view(-1, 1, 32, 32).cuda()
out1,out2,out3 =None,None,None

for cnn in cnnList:
    # load network
    model = defineModel(cnn)
    weight_dir = "./models/liberty_train_{0}_{1}/_liberty_min_as_fliprot/checkpoint_19.pth".format(cnn, batch)
    # Load weights
    trained_weight = torch.load(weight_dir)
    model.load_state_dict(trained_weight['state_dict'])
    # Forward pass the patches to the model
    output = model(patches)

    # Resize, output result
    print("original size ", output.shape)
#     output.shape = torch.Size([1000, 128])
    out = output.view(5, 200, 128).cpu().data
    print("resize: ", out.shape)
    outList.append(out)

# Repeat above process for CNN2, CNN3, and store them together:
all_output = torch.stack((outList[0], outList[1], outList[2]))
# all_output = torch.stack((outList[0],outList[0]))
torch.save(all_output, output_dir)
test_patches = torch.load(output_dir)
print(type(test_patches))
print("all ouput: ", test_patches.shape)


original size  torch.Size([1000, 128])
resize:  torch.Size([5, 200, 128])
original size  torch.Size([1000, 128])
resize:  torch.Size([5, 200, 128])
original size  torch.Size([1000, 128])
resize:  torch.Size([5, 200, 128])
<class 'torch.Tensor'>
all ouput:  torch.Size([3, 5, 200, 128])
